## Set your API Endpoint

In [ ]:
import autogen
from pathlib import Path

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-0125-preview", "gpt-3.5-turbo"],
    },
)

## Creating Tools

In [ ]:
import requests
import time

# API base URL (str)
api_base_url = "https://api.coingecko.com/api/v3/"

# Rate limit (int) and delay between requests (float)
rate_limit = 30  # 30 requests per minute for public API
delay_between_requests = 60 / rate_limit

def handle_api_errors(response: requests.Response) -> None:
  """Handles API errors based on status code

  Args:
      response: The API response object
  """
  status_code = response.status_code
  if status_code == 400:
      print("Bad Request. Please check your request and try again.")
  elif status_code == 401:
      print("Unauthorized. Check your API key and credentials.")
  elif status_code == 403:
      print("Forbidden. Your access is blocked. Please contact CoinGecko support.")
  elif status_code == 429:
      print("Rate limit exceeded. Please reduce the number of requests or consider upgrading to a paid plan.")
  elif status_code == 500:
      print("Internal Server Error. Please try again later.")
  elif status_code == 503:
      print("Service Unavailable. Check the API status and updates on https://status.coingecko.com/")

from typing import Optional

def make_api_request(url: str, params: Optional[dict[str, str]] = None) -> dict[str, str] | None:
  """Makes an API request to the CoinGecko API with rate limiting and error handling (CoinGecko is a cryptocurrency data provider)

    Args:
      url: The API endpoint URL
      params: Optional dictionary of query parameters

  Returns:
      A dictionary containing the API response data, or None if there was an error
  """
  response = requests.get(url, params=params)
  if response.status_code != 200:
      handle_api_errors(response)
      return None
  time.sleep(delay_between_requests)  # Respect rate limit
  return response.json()


def get_total_market_cap():
  """Fetches the total market capitalization for the cryptocurrency market

  Returns:
      The total market cap in USD (float), or None if there was an error
  """
  url = f"{api_base_url}global"
  data = make_api_request(url)
  if data:
      total_market_cap = data["data"]["total_market_cap"]["usd"]
      return total_market_cap
  return None


def get_top_gainers(limit: int = 10) -> list | None:
  """Fetches the top gainers over the last 24 hours for the cryptocurrency market

  Args:
      limit: The number of top gainers to fetch (default: 10)

  Returns:
      A list of dictionaries containing information about the top gainers, 
      or None if there was an error
  """
  url = f"{api_base_url}coins/markets"
  params = {
      "vs_currency": "usd",
      "order": "market_cap_desc",
      "per_page": limit,
      "page": 1,
  }
  data = make_api_request(url, params)
  if data:
      top_gainers = []
      for coin in data:
          top_gainers.append(
              {
                  "name": coin["name"],
                  "symbol": coin["symbol"],
                  "price_change_percentage_24h": coin["price_change_percentage_24h"],
              }
          )
      return top_gainers
  return None


def get_top_losers(limit: int = 10) ->  list | None:
  """Fetches the top losers over the last 24 hours for the cryptocurrency market

  Args:
      limit: The number of top losers to fetch (default: 10)

  Returns:
      A list of dictionaries containing information about the top losers, 
      or None if there was an error
  """
  url = f"{api_base_url}coins/markets"
  params = {
      "vs_currency": "usd",
      "order": "market_cap_asc",
      "per_page": limit,
      "page": 1
  }
  data = make_api_request(url, params)
  if data:
      top_losers = []
      for coin in data:
          top_losers.append({
              "name": coin["name"],
              "symbol": coin["symbol"],
              "price_change_percentage_24h": coin["price_change_percentage_24h"]
          })
      return top_losers
  return None

def get_trading_volume(coin_ids: list[str], vs_currencies: str = "usd") -> dict[str, float] | None:
  """Fetches the trading volume for a list of coin IDs, for the cryptocurrency market

  Args:
      coin_ids: A list of coin IDs (str)
      vs_currencies: The currency to compare prices against (default: "usd")

  Returns:
      A dictionary mapping coin IDs to their 24 hour trading volume in the specified currency (float), 
      or None if there was an error
  """
  url = f"{api_base_url}simple/price"
  params = {
      "ids": ",".join(coin_ids),
      "vs_currencies": vs_currencies,
      "include_24hr_vol": True,
  }
  data = make_api_request(url, params)
  if data:
      trading_volume = {}
      for coin_id, volume in data.items():
          trading_volume[coin_id] = volume["usd"]
      return trading_volume
  return None

## Construct Agents

In [ ]:
gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

user_proxy = autogen.UserProxyAgent(
    name="Client",
    system_message="A human client. Interact with the CEO to discuss your Crypto report needs.",
    code_execution_config=False,
)

ABS_PATH: Path = Path(__file__).parent.absolute()
agency_system_message = ABS_PATH.joinpath("instructions/agency_system_message.md").open("r").read()
ceo_system_message = ABS_PATH.joinpath("instructions/agents/systems/ceo.md").open("r").read()
marketAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/marketAnalyst.md").open("r").read()
fundamentalResearchAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/fundamentalResearchAnalyst.md").open("r").read()
investmentResearchAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/investmentResearchAnalyst.md").open("r").read()
riskManagementSpecialist_system_message = ABS_PATH.joinpath("instructions/agents/systems/riskManagementSpecialist.md").open("r").read()
socialMediaAnalyst_system_message = ABS_PATH.joinpath("instructions/agents/systems/socialMediaAnalyst.md").open("r").read()

ceo = autogen.ConversableAgent(
    name="CEO",
    llm_config=gpt4_config,
    system_message=ceo_system_message + "\n\nUse the following as the corporate official guide:\n\n" + agency_system_message,
)

marketAnalyst = autogen.ConversableAgent(
    name="Market Analyst",
    llm_config=gpt4_config,
    system_message=marketAnalyst_system_message,
)

fundamentalResearchAnalyst = autogen.ConversableAgent(
    name="Fundamental Research Analyst",
    llm_config=gpt4_config,
    system_message=fundamentalResearchAnalyst_system_message,
)

investmentResearchAnalyst = autogen.ConversableAgent(
    name="Investment Research Analyst",
    llm_config=gpt4_config,
    system_message=investmentResearchAnalyst_system_message,
)

riskManagementSpecialist = autogen.ConversableAgent(
    name="Risk Management Specialist",
    llm_config=gpt4_config,
    system_message=riskManagementSpecialist_system_message,
)

socialMediaAnalyst = autogen.ConversableAgent(
    name="Social Media Analyst",
    llm_config=gpt4_config,
    system_message=socialMediaAnalyst_system_message,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, ceo, marketAnalyst, fundamentalResearchAnalyst, investmentResearchAnalyst, riskManagementSpecialist, socialMediaAnalyst], messages=[], max_round=50
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

## Start Chat

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
I want a general report on the crypto market for today.
""",
)